# Linear Systems
## A review for Electrical Systems Load Flow

### Linear System definition
Let a linear system of $n$ variables and $m$ equations represented by:
$$
\begin{equation}
\tag{1}
\begin{cases}
a_{11}x_1 + a_{12}x_2 + \dotsb + a_{1n}x_n = b_1 \\
a_{21}x_1 + a_{22}x_2 + \dotsb + a_{2n}x_n = b_2 \\
\qquad \vdots \\
a_{m1}x_1 + a_{m2}x_2 + \dotsb + a_{mn}x_n = b_m
\end{cases}
\end{equation}
$$
Where $x_j$ are the variables, $a_{ij}$ are the coefficients and $b_{i}$ are the constants.

In matrix form, these variables, coefficients and constants can be represented as:
$$
\begin{equation}
\tag{2}
Ax=B
\end{equation}
$$
Where $A$ is the coefficient matrix, with $m$ by $n$ dimensions, $x$ is the variables vector, with a length of $n$, and $B$ is the constants vector, with a length of $m$, whose elements are represented by:
$$
\begin{equation}
\tag{3}
\begin{bmatrix}
a_{11} & a_{12} & \dotsb & a_{1n} \\
a_{21} & a_{22} & \dotsb & a_{2n} \\
\vdots & \vdots & \ddots & \vdots \\
a_{m1} & a_{m2} & \dotsb & a_{mn}
\end{bmatrix}
\begin{bmatrix}
x_{1} \\
x_{2} \\
\vdots \\
x_{n}
\end{bmatrix}
=
\begin{bmatrix}
b_{1} \\
b_{2} \\
\vdots \\
b_{m}
\end{bmatrix}
\end{equation}
$$
A system is said triangular when the number of equations is equal to the number or variables, therefore, $m=n$.

### Matrix solution of linear systems

In matrix form, a linear system can be solved by left-multiplying the expression:
\begin{equation}
\tag{2}
Ax=B
\end{equation}
by the inverse of the $A$ matrix:
\begin{equation}
\tag{4}
A^{-1}Ax=A^{-1}B
\end{equation}
Since:
\begin{equation}
\tag{5}
A^{-1}A = I
\end{equation}
and the identity matrix is multiplication's neutral element, therefore the variables vector can be defined as:
\begin{equation}
\tag{6}
x=A^{-1}B
\end{equation}

### Iterative solution of linear systems using Jacobi's method (or Gauss' method)

A linear system in the matrix form $Ax=b$ can be solved by using fixed point iterations.

Jacobi's method (or Guass') applied to a system of $n$ variables, consists on isolating, in the $i$-th equation, the variable $x_{i}$ and calculating the value of its $k$-th iteration using the values from the $(k-1)$-th iteration of the other variables.

Assuming $x^{(1)}$ as a vector with the initial estimate of the linear system's variables, its $i$-th equation has a general form:
$$
\begin{equation}
\tag{7}
x_{i}^{(k)}=\left(b_{i} - \sum_{j=1}^{j=i-1} a_{ij} x_j^{(k-1)} - \sum_{j=i+1}^{j=n} a_{ij} x_j^{(k-1)} \right) \cdot \left( a_{ii} \right)^{-1}
\end{equation}
$$
It is assumed that the $A$ matrix is non-singular (invertible).

The method will converge when the infinity norm of the difference between the results of the $k$-th and the $(k-1)$-th iteration are less than an assumed tolerance. That is, when the highest of the differences for all the variables is less than this tolerance.
$$
\begin{equation}
\tag{8}
\left\lVert x^{(k)} - x^{(k+1)} \right\rVert_\infty = \max_{i=1}^{i=n} \left| x_i^{(k)} - x_i^{(k+1)} \right|
\end{equation}
$$

In [2]:
def gauss(A , b , x0 , tol , it_max):
    """
    Método de Jacobi/Gauss para solução de sistemas lineares de forma iterativa
    Entradas:
        A: matriz dos coeficientes (nxn)
        b: vetor das constantes (nx1)
        x0: vetor das aproximações iniciais das variáveis (nx1)
            no processo iterativo, este vetor armazenará os resultados da iteração anterior
        tol: tolerância esperada das soluções
        it_max: número máximo de iterações
    Saídas:
        x: vetor da solução do sistema dentro da tolerância estabelecida (nx1)
        it: número de iterações para chegar ao resultado de x
    Reproduzido de Justo et al (2020, p.123) com a adição de comentários para esclarecimentos didáticos futuros
    Referência bibliográfica:
    JUSTO, D. A. R. et al. Cálculo Numérico - Um Livro Colaborativo Versão Python. 19 de agosto de 2020. Disponível em: https://github.com/reamat/CalculoNumerico
    """
    # Garantindo que as matrizes/vetores do sistema sejam do tipo double
    A - A.astype("double")
    b = b.astype("double")
    x0 = x0.astype("double")

    ## TODO:
    # 1) Valores padrão para tol e it_max
    # 2) Garantir que A seja singular e triangular

    # Determina o número de variáveis pelo tamanho da matriz A
    n = np.shape(A)[0]
    # Cria um vetor de zeros para armazenar as soluções a cada iteração
    x = np.zeros((n,1))
    # Inicializa o contador de iterações
    it = 0

    # Laço principal para que o método não itere mais vezes do que o estabelecido
    while it < it_max:
        it += 1 # Contador de iterações
        for e in np.arange(n):
            # Inicializa o vetor das soluções com o valor da constante da linha atual
            x[e] = b[e]
            # Executa o somatório
            for i in np.concatenate((np.arange(0 , e) , np.arange(e+1 , n))):
                # Somatório de -A(i,j)*x0(j), onde x0(j) é o valor anterior
                x[e] -= A[e , i] * x0[i]
            # Depois do somatório, divide tudo por A(i,i)
            x[e] /= A[e , e]
        # Norma do máximo para determinar se a maior tolerância é menor que aquela escolhida
        if (np.linalg.norm(x - x0 , np.inf) < tol):
            # Se for, sai da função retornando o valor das variáveis (x) e o número de iterações (it)
            return x , it
        # Copia o resultado da iteração atual para que usar na próxima iteração como valor anterior
        x0 = np.copy(x)
    # Caso saia do laço por estourar o número de iterações, dá erro.
    raise NameError('The maximum number of iterations has been met.')

### Iterative solution of linear systems using Gauss-Seidel's method

Gauss-Seidel's method consists in a modification of Jacobi's method in which, on calculating the $i$-th equation on its $k$-th iteration, the values used, for all the variables from $x_1$ to $x_{i-1}$, are from the current iteration ($k$) and not the previous iteration ($k-1$). The values for variables $x_{i+1}$ to $x_n$ are still from the $(k-1)$-th iteration.

Therefore, the general form for Gauss-Seidel's method is:
$$
\begin{equation}
\tag{9}
x_{i}^{(k)}=\left(b_{i} - \textcolor{blue}{\sum_{j=1}^{j=i-1} a_{ij} x_j^{(k)}} - \sum_{j=i+1}^{j=n} a_{ij} x_j^{(k-1)} \right) \cdot \left( a_{ii} \right)^{-1}
\end{equation}
$$
It is assumed that the $A$ matrix is non-singular (invertible).

This method's converge is the same as Jacobi's method.

In [3]:
def gaussseidel(A , b , x0 , tol , it_max):
    """
    Método de Gauss-Seidel para solução de sistemas lineares de forma iterativa
    Entradas:
        A: matriz dos coeficientes (nxn)
        b: vetor das constantes (nx1)
        x0: vetor das aproximações iniciais das variáveis (nx1)
            no processo iterativo, este vetor armazenará os resultados da iteração anterior
        tol: tolerância esperada das soluções
        it_max: número máximo de iterações
    Saídas:
        x: vetor da solução do sistema dentro da tolerância estabelecida (nx1)
        it: número de iterações para chegar ao resultado de x
    Reproduzido de Justo et al (2020, p.125) com a adição de comentários para esclarecimentos didáticos futuros
    Referência bibliográfica:
    JUSTO, D. A. R. et al. Cálculo Numérico - Um Livro Colaborativo Versão Python. 19 de agosto de 2020. Disponível em: https://github.com/reamat/CalculoNumerico
    """
    # Garantindo que as matrizes/vetores do sistema sejam do tipo double
    A - A.astype("double")
    b = b.astype("double")
    x0 = x0.astype("double")

    ## TODO:
    # 1) Valores padrão para tol e it_max
    # 2) Garantir que A seja singular e triangular

    # Determina o número de variáveis pelo tamanho da matriz A
    n = np.shape(A)[0]
    # Cria um vetor das soluções a cada iteração copiando o vetor dos resultados da iteração anterior
    x = np.copy(x0)
    # Inicializa o contador de iterações
    it = 0

    # Laço principal para que o método não itere mais vezes do que o estabelecido
    while it < it_max:
        it += 1 # Contador de iterações
        for e in np.arange(n):
            # Inicializa o vetor das soluções com o valor da constante da linha atual
            x[e] = b[e]
            # Executa o somatório
            for i in np.concatenate((np.arange(0 , e) , np.arange(e+1 , n))):
                # Somatório de -A(i,j)*x(j), onde x é o vetor dos resultados
                # Este vetor, na k-éstima iteração terá os elementos entre 0 e k-1 com resultado calculado na iteração atual
                # e os elementos entre k+1 e n com o resultado da iteração anterior
                x[e] -= A[e , i] * x[i]
            # Depois do somatório, divide tudo por A(i,i)
            x[e] /= A[e , e]
        # Norma do máximo para determinar se a maior tolerância é menor que aquela escolhida
        if (np.linalg.norm(x - x0 , np.inf) < tol):
            # Se for, sai da função retornando o valor das variáveis (x) e o número de iterações (it)
            return x , it
        # Copia o resultado da iteração atual para que usar na próxima iteração como valor anterior
        x0 = np.copy(x)
    # Caso saia do laço por estourar o número de iterações, dá erro.
    raise NameError('The maximum number of iterations has been met.')

In [4]:
from __future__ import division
import numpy as np

A = np.array([[3 , 1 , -1] , [-1 , -4 , 1] , [1 , -2 , -5]])
b = np.array([[2] , [-10] , [10]])
x0 = np.array([[1] , [1] , [1]])
tol = 1e-5
it_max = 100

resultado = np.linalg.solve(A , b)
(resultado1 , it1) = gauss(A , b , x0 , tol , it_max)
(resultado2 , it2) = gaussseidel(A , b , x0 , tol , it_max)

print(f"Linear system methods comparison\n")
print(f"1) Using the numpy.linalg.solve function (LU decomposition):")
print(resultado)
print(f"\n")
print(f"2) Using Gauss' method:")
print(f"number of iterations: {it1}")
print(resultado1)
print(f"\n")
print(f"3) Using Gauss-Seidel's method:")
print(f"number of iterations: {it2}")
print(resultado2)

Linear system methods comparison

1) Using the numpy.linalg.solve function (LU decomposition):
[[-1.]
 [ 2.]
 [-3.]]


2) Using Gauss' method:
number of iterations: 12
[[-0.99999671]
 [ 1.99999954]
 [-2.99999738]]


3) Using Gauss-Seidel's method:
number of iterations: 8
[[-1.00000149]
 [ 1.99999987]
 [-3.00000025]]
